Install packages and generate the mesh

In [1]:
# Import packages
import netgen.gui
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from xfem import *
from netgen.geom2d import SplineGeometry
import numpy as np


# Generate mesh
pipe = SplineGeometry()
pipe.AddRectangle([-5,0],[5,4],  bcs = ("wall" , "outlet", "wall", "inlet"))
mesh = Mesh(pipe.GenerateMesh(maxh=0.3, quad_dominated=False))

Create a level set function

$\phi = \sqrt{x^2+y^2}-1$

to indicate cell boundary

In [2]:
# Level set function
levelset = (sqrt(x*x+y*y) - 1.0)


# Interpolate level set function to piecewise linear function
lsV = H1(mesh,order=1)
lsetp1 = GridFunction(H1(mesh,order=1))
lsetp1_old = GridFunction(lsV)
InterpolateToP1(levelset,lsetp1)

Draw(lsetp1,mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-64-gea137ded', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

Definie some constants

In [3]:
# Normal direction
n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)


# Mesh size
h = specialcf.mesh_size


# Viscosity, elasticity constants and density
mu = 1
lambd = 1
rho = 1
gamma = 20


# Stabilization parameter for ghost-penalty
gamma_stab_vel = 0.05  # if set to zero: no GP-stabilization for velocity
gamma_stab_dis = 0.05
    
    
# Boundary condition for the displacement
disp_D = (0,0)

Create the FEM spaces

$$V_h = \{v \in [H_0^1(\Omega)]^2|v|_T \in [\mathcal{P}^2(T)]^2, T\in \mathcal{T}_h\}$$

and set the boundary conditions
    $$ u = 0  \text{ on } \partial\Omega\cap\partial\Omega_-$$


In [4]:
# CutFEM space
order = 2
ci = CutInfo(mesh, lsetp1)
Vhbase = VectorH1(mesh, order=order, dirichlet="wall")
Vh = Compress(Vhbase, GetDofsOfElements(Vhbase, ci.GetElementsOfType(HASNEG)))
VhG = FESpace([Vh,Vh],dgjumps=True)
freedofs = VhG.FreeDofs()


# Set boundary conditions
gfuold = GridFunction(VhG)
gfu = GridFunction(VhG)
vel_old, disp_old = gfuold.components #velocity and displacement of the previous time step 
vel, disp = gfu.components #velocity and displacement
disp.Set(disp_D, definedon=mesh.Boundaries("wall"))


# Draw velocity or levelset function
animation = GridFunction(VhG).components[1]
animation = IfPos(-lsetp1, disp, (0,0))
#scene = Draw (animation, mesh)
scene = Draw(lsetp1,mesh)
 

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-64-gea137ded', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

Timestepping

In [5]:
dt = 0.05
for i in range(2):
    t = i*dt
    
    
    # Update 
    gfuold = gfu
    lsetp1_old = lsetp1
    disp.Set(disp_D, definedon=mesh.Boundaries("wall") )

    
    # Normal direction
    n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
    
    
    # Level set domains
    lset_neg = { "levelset" : lsetp1, "domain_type" : NEG}
    lset_pos = { "levelset" : lsetp1, "domain_type" : POS}
    lset_if  = { "levelset" : lsetp1, "domain_type" : IF }


    # Facets on which ghost penalty stabilization should be applied
    hasif = ci.GetElementsOfType(IF)
    hasneg = ci.GetElementsOfType(HASNEG)
    ba_facets = GetFacetsWithNeighborTypes(mesh,a=hasneg,b=hasif, use_and = True)
    
    
    # Test and trial functions
    v,u = VhG.TrialFunction()
    x,w = VhG.TestFunction()
    
    
    # Some helper expressions:
    I = Id(mesh.dim)
    F = I-Grad(u)
    J = Det(F)
    Finv = Inv(F)
    FinvT = Inv(F.trans)
    def eps(u):
        return 0.5*(Grad(u) + Grad(u).trans)
    def sigma(u):
        return J*Finv*(2*mu*eps(u) + lambd* div(u)*I)*FinvT
    

Generate a 'bilinear' form $b=a-f$ with

$$ a(v,u,x,w) = (v-velold,w)_{L^2(\Omega_-(t))}+dt\left((\rho v \cdot\nabla v,w)_{L^2(\Omega_-(t))}+(\sigma(u),\epsilon(w))_{L^2(\Omega_-(t))}\right)$$
$$ + (u-dispold,x)_{L^2(\Omega_-(t))}-dt(v+v\cdot\nabla u,x)_{L^2(\Omega_-(t))}$$
and $$f(w) = dt (f,w)_{L^2(\Omega_-(t))}+dt (g,w)_{L^2(\partial\Omega_-(t))}$$

In [6]:
    # Bilinearform a
    bfi_neg = SymbolicBFI(levelset_domain = lset_neg, form = dt*(rho*(grad(v)*v)*w+InnerProduct(sigma(u),eps(w))-v*x+(grad(u)*v)*x))
    bfi_der = SymbolicBFI(levelset_domain = lset_neg, form = InnerProduct(v-vel_old,w)+InnerProduct(u-disp_old,x))
    
    
    # Linearform f
    f_coef = CoefficientFunction((10,0)) # Volume force
    g = CoefficientFunction((0,0)) #Surface force
    lfi_neg = SymbolicBFI(levelset_domain = lset_neg, form = dt*f_coef*w)
    lfi_if = SymbolicBFI(levelset_domain = lset_if, form = dt*g*w) 
    
    
    # Ghost penalty terms:
    if gamma_stab_vel > 0:
        gp_vel= SymbolicFacetPatchBFI(form = - gamma_stab_vel*(x-x.Other())*(v-v.Other()),skeleton=False,definedonelements=ba_facets)
    if gamma_stab_dis > 0:
        gp_dis= SymbolicFacetPatchBFI(form = - gamma_stab_dis/h**2*(u-u.Other())*(w-w.Other()),skeleton=False,definedonelements=ba_facets)

        
    # Combine bilinear and linear form
    b = BilinearForm(VhG, symmetric = False)
    b += bfi_neg
    b += bfi_der
    b += lfi_neg
    b += lfi_if
    #b += gp_vel
    #b += gp_dis
           
        
    # Assembly
    bfi_neg.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
    bfi_der.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
    lfi_neg.SetDefinedOnElements(ci.GetElementsOfType(HASNEG))
    lfi_if.SetDefinedOnElements(ci.GetElementsOfType(IF))
    b.Assemble()

    
    # Newton to linearize non-linear part
    Newton(b, gfu, dampfactor= 1)
    
    
    # Redraw velocity or levelset function
    #animation = IfPos(-lsetp1, gfu, (0,0))    
    scene.Redraw()
    print("time :", i*dt)
    
    

Newton iteration  0
err =  0.15744828766308833
Newton iteration  1
err =  0.0021393279039677838
Newton iteration  2
err =  0.0003068167289276351
Newton iteration  3
err =  1.2420757008424347e-06
Newton iteration  4
err =  2.3491129187170103e-11
Newton iteration  5
err =  7.52796871025003e-17
time : 0.05


To calculate the new level set function solve the PDE
$$\frac{d\phi}{dt} = \partial_t\phi + u\cdot \nabla \phi = 0$$
Bilinear form:
    \begin{align*}
        a(\phi,v)=(\frac{\partial}{\partial t}\phi,v)_{L^2(\Omega)}+(u\cdot\nabla\phi,v)_{L^2(\Omega)}=0
    \end{align*}
    Streamline-Diffusion:
    \begin{align*}
        b(\phi,v) = ((\partial_t\phi+u\cdot \nabla \phi)(u\cdot\nabla v))_{L^2(\Omega)}
    \end{align*}
    
The timestep is performed using implicit Euler

In [7]:
    # Calculate new level set function
    phi, psi = lsV.TnT()
    eps = 0.00000000001 #to avoid dividing by zero if Norm(disp)=0
    c = BilinearForm(lsV, symmetric = False)
    c += (disp*grad(phi))*(psi+gamma*h/(Norm(disp)+eps)*disp*grad(psi))*dx
    c.Assemble()

    m = BilinearForm(lsV, symmetric = False)
    m += phi*(psi+gamma*h/(Norm(disp)+eps)*disp* grad(psi))*dx
    m.Assemble()

    mstar = m.mat.CreateMatrix()
    mstar.AsVector().data = m.mat.AsVector() + dt * c.mat.AsVector()
    invmstar = mstar.Inverse(freedofs=lsV.FreeDofs(),inverse="umfpack")

    res = lsetp1.vec.CreateVector()
    res.data =  - dt * c.mat * lsetp1.vec
    lsetp1.vec.data += invmstar *res 
    
    # Update Cutinfo
    ci.Update(levelset=lsetp1)


Draw(vel,mesh, 'velocity')
Draw(disp,mesh, 'displacement')
Draw(animation,mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-64-gea137ded', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-64-gea137ded', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2008-64-gea137ded', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …